# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [123]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os


In [104]:
# Remove
gkey = ""
gmaps.configure(api_key = gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [141]:
# To read .csv back in...
weather_batch_all = pd.read_csv("..\WeatherPy\output_data\city_weather.csv", index_col = 0)
weather_batch_all.head(50)

city  latitude     longitude  max_temp  humidity  \
0           Kingisepp     58.92  2.854000e+01     32.90      93.0   
1         Provideniya     53.36 -1.769900e+02     39.97      59.0   
2          Negru voda     43.61  2.822000e+01     45.00      93.0   
3             Paamiut     55.70 -5.055000e+01     32.11      77.0   
4              Barrow     84.17 -1.438900e+02     28.40      69.0   
5             Huaihua     27.51  1.097100e+02     65.62      96.0   
6             Mumford      3.81 -1.637090e-10     81.00      85.0   
7       Pacific grove     30.61 -1.313900e+02     57.00      86.0   
8        Punta arenas    -69.78 -1.123600e+02     42.80      93.0   
9              Visnes     57.99  2.050000e+00     41.00      70.0   
10              Kieta      0.79  1.602800e+02     84.24      67.0   
11              Tiksi     82.24  1.271000e+02     16.84      81.0   
12            Rikitea    -28.88 -1.409900e+02     79.03      72.0   
13          Cape town    -63.90 -1.100000e-01     59.00      82.0   
14      Mar del plata    -46.49 -4.741000e+01     66.20      72.0   
15              Prado    -17.73 -3.667000e+01     74.79      85.0   
16            Mataura    -55.83 -1.498500e+02     50.00      84.0   
17             Atuona      9.99 -1.301900e+02     80.38      78.0   
18              Bluff    -67.99  1.714700e+02     50.00      84.0   
19           La ligua    -32.28 -7.131000e+01     56.64      87.0   
20             Dingle     50.43 -1.968000e+01     89.60      53.0   
21          Lugovskoy     56.96  6.441000e+01     44.60      42.0   
22               Lebu    -32.66 -1.001700e+02     52.45      91.0   
23                Vao    -28.14  1.654500e+02     75.92      74.0   
24         Butaritari     18.81  1.643600e+02     81.37      78.0   
25              Ruiru     -1.15  3.700000e+01     64.40     100.0   
26    Arraial do cabo    -26.64 -3.648000e+01     73.72      77.0   
27         Talcahuano    -30.63 -8.368000e+01     55.40      94.0   
28       Saint george     27.01 -5.623000e+01     78.80       9.0   
29           Ratodero     27.79  6.824000e+01     73.40      60.0   
30              Boiro     42.53 -8.990000e+00     59.00      94.0   
31  Sao joao da barra    -28.32 -3.251000e+01     69.24      87.0   
32               Hilo     15.19 -1.513300e+02     78.80      65.0   
33           Khatanga     73.21  1.032700e+02     33.26      96.0   
34             Bethel     48.97 -1.659800e+02     57.20      24.0   
35            Klyuchi     55.38  1.609700e+02     38.57      62.0   
36               Ayan     54.97  1.382400e+02     34.41      64.0   
37         Westerland     54.76  8.580000e+00     41.00      85.0   
38       Jacareacanga     -6.73 -5.877000e+01     74.37      97.0   
39              Mugan     40.34  4.910000e+01     50.74      44.0   
40            Ushuaia    -74.89 -5.256000e+01     42.80      93.0   
41          Fairbanks     64.12 -1.462100e+02     57.99      21.0   
42        East london    -87.38  5.959000e+01     62.29      75.0   
43              Kapaa     15.36 -1.699300e+02     77.00      73.0   
44         Aromashevo     56.82  6.851000e+01     40.62      55.0   
45           Hermanus    -89.01 -1.369000e+01     55.00      89.0   
46            Butembo      0.20  2.824000e+01     56.01      95.0   
47              Pevek     84.40  1.699600e+02     18.66      85.0   
48               Nome     71.72 -1.659300e+02     44.60      49.0   
49            Isangel    -20.40  1.741300e+02     86.00      70.0   

    cloud_cover  wind_speed country                 date  
0          99.0        5.64      RU  2021-04-24 01:47:24  
1          13.0        4.50      RU  2021-04-24 01:47:25  
2           0.0        3.44      RO  2021-04-24 01:47:25  
3           2.0        4.52      GL  2021-04-24 01:37:53  
4           1.0        5.12      US  2021-04-24 01:43:14  
5         100.0        4.07      CN  2021-04-24 01:47:26  
6         100.0        9.91      GH  2021-04-24 01:47:26  
7          44.0        6.

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [106]:
# Access maps with unique API key
gmaps.configure(api_key = gkey)

In [107]:
# Store latitude and longitude in locations
locations = weather_batch_all[["latitude", "longitude"]]
humidity = weather_batch_all["humidity"]

# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}


# Plot Heatmap
fig = gmaps.figure(center = (0, 0), zoom_level = 1.65, layout = figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights = humidity, 
                                 dissipating = True, 
                                 max_intensity = 200,
                                 point_radius = 40,
                                 opacity = 0.3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [116]:
# Let's head to the rain forest! Hot and wet!

hot_and_wet = weather_batch_all.loc[(weather_batch_all["humidity"] > 80) &
                                    (weather_batch_all["wind_speed"] < 20.0) &
                                    (weather_batch_all["cloud_cover"] > 50) &
                                    (weather_batch_all["max_temp"] > 80.0)]

hot_and_wet.head()

city  latitude     longitude  max_temp  humidity  cloud_cover  \
6     Mumford      3.81 -1.637090e-10     81.00      85.0        100.0   
66     Harper     -4.06 -6.700000e+00     80.62      83.0         84.0   
161     Tabou     -3.84 -5.030000e+00     80.02      84.0         94.0   
211   Caucaia     -1.79 -3.714000e+01     82.00      89.0         75.0   
352  Acapulco      5.59 -1.031500e+02     80.60      83.0         90.0   

     wind_speed country                 date  
6          9.91      GH  2021-04-24 01:47:26  
66         7.90      LR  2021-04-24 01:48:43  
161        7.27      CI  2021-04-24 01:51:12  
211        3.44      BR  2021-04-24 01:53:28  
352        9.22      MX  2021-04-24 01:59:40

In [159]:
hotel_df = hot_and_wet.copy()
hotel_df.reset_index(drop = True, inplace = True)
hotel_df.head()

city  latitude     longitude  max_temp  humidity  cloud_cover  \
0   Mumford      3.81 -1.637090e-10     81.00      85.0        100.0   
1    Harper     -4.06 -6.700000e+00     80.62      83.0         84.0   
2     Tabou     -3.84 -5.030000e+00     80.02      84.0         94.0   
3   Caucaia     -1.79 -3.714000e+01     82.00      89.0         75.0   
4  Acapulco      5.59 -1.031500e+02     80.60      83.0         90.0   

   wind_speed country                 date  
0        9.91      GH  2021-04-24 01:47:26  
1        7.90      LR  2021-04-24 01:48:43  
2        7.27      CI  2021-04-24 01:51:12  
3        3.44      BR  2021-04-24 01:53:28  
4        9.22      MX  2021-04-24 01:59:40

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [166]:
hotel_df.loc[0, "latitude"]
lat = weather_batch_all.loc[41, "latitude"]
lon = weather_batch_all.loc[41, "longitude"]
test_string = f"{round(lat, 5)}, {round(lon, 5)}"
print(test_string)

64.12, -146.21


In [167]:
# geocoordinates
target_coordinates = test_string
target_radius = 50000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [168]:
# convert response to json
places_data = response.json()

# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "next_page_token": "ATtYBwKsqkxkxrX0Yy-iGNo7inGGmATbGJafnMETSLL0RZXmG2dXhb1_DOppzIUYyeqf0G_z6rTLvYU7ngyn7TuddFke084bQ-peWHq_UqtKKjVY22bivtIF1joGx-Rwf_XvwTV_7MHoS1s4HUJKE9rjt9LAwt2XL0EG3CeIneLe51JGuUx3ZTPCzSHM5Wzwc3suqT86WvCi2B2iar-o9XN_awQiYyKo90gOHxqwwZhwHOzJ97z3shtITbYAEQH3H_FPAAxZtSAkJ9yV4cBlRQkf2ZdIi6F5ZpRhp5e2F8Z2wHGHMrPYwOaywlwNaXRv99WHBl18EU8ZHE42kmZuB8Cn1bIMiW8VAd_2iQH2hDcwm-wAnN2cZQq6buteK2X7ir0rgZ1yQm-l7H_ps6JrPGxiuLV9GDTTojIjIjjXAlG2JHrALmYlCU3rGrs",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 64.0407439,
                    "lng": -145.7337224
                },
                "viewport": {
                    "northeast": {
                        "lat": 64.0420720302915,
                        "lng": -145.7325540197085
                    },
                    "southwest": {
                        "lat": 64.03937406970849,


In [165]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
